In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
#import foolbox
import tqdm
import imageio
from torchvision import transforms
from torch.autograd import Variable
import torch.nn as nn
import os
from torch.utils.data import Dataset,DataLoader
import skimage.transform
import skimage
from skimage import io
import cv2
from torch.nn import functional as F
import copy

In [2]:
torch.cuda.set_device(3)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#calcyulating the mean and the standard deviation


In [4]:
#model = torchvision.models.resnet18(pretrained=True, )

In [5]:
#model.eval()
#_=model.to(DEVICE)

In [6]:
class ResnetNetwork(nn.Module):
    def __init__(self):
        super(ResnetNetwork, self).__init__()
        cnn = torchvision.models.resnet152(pretrained=True)
        #lastlayer_in = cnn.fc.in_features
        #cnn.fc = nn.Linear(lastlayer_in, 1000)
        self.sim_model = cnn
    def forward(self, input1 = None):
        output1 = self.sim_model(input1)
        #output2 = self.fc1(output1)
        return output1

In [7]:
mean = np.array([0.5, 0.5, 0.5]).reshape((3, 1, 1))
std = np.array([0.5, 0.5, 0.5]).reshape((3, 1, 1))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(mean=(0.485, 0.456, 0.406),std= (0.229, 0.224, 0.225))
    ]
)


In [8]:
class cam_dataset(Dataset):
    def __init__(self,img_transform1):
        #/workspace/data_fine_grain/cub200/raw/CUB_200_2011/(base) ironman@bigbox:~$ CUB_200_2011
        self.dir_path = './test/'
        self.check_path='./imagenet_fgsm/imagenet/fgsm/'
        self.files_list = []
        self.original_label = []
        self.misclassified_label = []
        self.img_transform1 = img_transform1
        self.original_image= []
        
        for r, d, f in os.walk(self.dir_path):
            for file in f:
                if '.png' in file and "-checkpoint.png" not in file:
                    #print("here")
                    #print(file)
                    #print(file.split('.')[-2].split('_')[-2])
                    #print(file.split('.')[-2].split('_')[-1])
                    #if(int(file.split('.')[-2].split('_')[-2])==int(file.split('.')[-2].split('_')[-1])):
                    #print(file)
                    self.files_list.append(os.path.join(r,file))
                    self.original_label.append(int(file.split('.')[-2].split('_')[-2]))
                    self.misclassified_label.append(int(file.split('.')[-2].split('_')[-1]))
                    self.original_image.append(os.path.join(self.check_path,file))
                    #print(os.path.join(self.check_path,file.split('.')[-2]+'.png'))

        #print(self.original_image)
            
    def __getitem__(self,index):
        #if(self.train):
            img2_path = self.original_image[index]
            img2_path = img2_path.rstrip()
            img2 = io.imread(img2_path)
            
            #print(img1.shape)
            if(img2.shape.__len__() == 2):
                img2 = np.repeat(img2[:, :, np.newaxis], 3, axis=2)
            img2 = img2.astype(np.float)
            img2 = cv2.resize(img2,(256,256),interpolation = cv2.INTER_AREA)
            img2 = img2/255.0
            #img2 = img2/255
           
            img2 = self.img_transform1(img2)
            #print("check the image")
            #print(np.unique(img2))
            #img2 = self.img_transform1(img2)
        
        
           
            img1_path = self.files_list[index]
            img1_path = img1_path.rstrip()
            img1 = io.imread(img1_path)
            
            #print(img1.shape)
            if(img1.shape.__len__() == 2):
                img1 = np.repeat(img1[:, :, np.newaxis], 3, axis=2) 
            #print(img1_path)
            #plt.imshow(img1)
            img1 = img1.astype(np.float)
            img1 = cv2.resize(img1,(256,256),interpolation = cv2.INTER_AREA)
            img1 = img1/255.0
           
            img1 = self.img_transform1(img1)
            label = self.original_label[index]
            label = int(label)
            mislabel = self.misclassified_label[index]
            mislabel = int(mislabel)
            return {"img": img1,"img2":img2,"label": label,"mislabel":mislabel,"path": img1_path}
   
        #return img1,img2,label
    def __len__(self):
        return len(self.original_label)


test_set = cam_dataset(transform)

testloader = DataLoader(test_set,batch_size =1,shuffle = True, num_workers =0)

In [9]:
data = test_set[0]

In [10]:
#print(data['img2'])

In [12]:
#plt.imshow(data["img"])
list_image = []
list_label = []
counter = 0
act_counter =0
for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    #####################actual image
    img2 = data['img2'].to(DEVICE)
    #path = data['path']
    img2 = img2.view(-1,img2.shape[-3],img2.shape[-2],img2.shape[-1])

    img2 = img2.float()
    resnet2 = ResnetNetwork()
    resnet2 = resnet2.to(DEVICE)
    resnet2.eval()
    img2 = Variable(img2.cuda(), requires_grad=True)
    pred2 = resnet2(img2)
    max_index2 = pred2.argmax(dim=1) 
    
    
    #print("check the original max index")
    #print(max_index2.item())
    list_image.append(data['path'])
    list_label.append(max_index2.item())
    #print("check the label")
    #print(label.item())
    
    #########image
    img1 = data['img'].to(DEVICE)
    path = data['path']
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])

    img1 = img1.float()
    
    
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    mislabel = data['mislabel'].to(DEVICE)
   
    resnet =  ResnetNetwork()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    #print("check the image sizes")
    #print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred = resnet(img1)
    max_index = pred.argmax(dim=1)
    val =nn.Softmax(dim=1)(pred)
    
    #print("check the max index of adv image")
    #print(max_index.item())
    #print("prob")
    #print(val.shape)
    values, index = torch.topk(val,1)
    #print(values.item())
    #print(index)
    #print("check the label")
    #print(label.item())
    #print(path)
    #break
    act_counter+=1
    if(max_index.item()!=max_index2.item()):
        print("wrong")
        print(path)
        print("original_label")
        print(max_index2.item()+1)
        print(max_index.item()+1)
        counter +=1
print("the final wrongs")
print(counter)
print("total")
print(act_counter)


   
    
  #print("the ")  
    

wrong
['./test/fgsm_n04367480_25111_840_840.png']
original_label
463
841
wrong
['./test/fgsm_n13133613_13488_998_998.png']
original_label
999
85
wrong
['./test/fgsm_n04367480_10144_840_840.png']
original_label
841
420
wrong
['./test/fgsm_n07711569_1558_935_935.png']
original_label
936
545
wrong
['./test/fgsm_n01631663_4835_27_27.png']
original_label
28
27
wrong
['./test/fgsm_n03710637_9473_638_638.png']
original_label
639
579
wrong
['./test/fgsm_n07711569_7523_935_935.png']
original_label
936
464
wrong
['./test/fgsm_n02102177_2388_218_218.png']
original_label
219
162
wrong
['./test/fgsm_n02102177_5418_218_218.png']
original_label
219
248
wrong
['./test/fgsm_n09468604_22268_979_979.png']
original_label
959
526
wrong
['./test/fgsm_n02454379_13804_363_363.png']
original_label
364
52
wrong
['./test/fgsm_n02102177_2695_218_218.png']
original_label
219
197
wrong
['./test/fgsm_n02102177_5597_218_218.png']
original_label
219
218
wrong
['./test/fgsm_n02219486_30176_310_310.png']
original_label


In [45]:
    '''
    ##############################################################for max_index
    pred[:,(max_index.item()-1)].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    
    img = io.imread(path[0])
    if(img.shape.__len__() == 2):
                img = np.repeat(img[:, :, np.newaxis], 3, axis=2) 
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    '''
    '''
    p = np.percentile(heatmap, 50)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    heatmap = heatmap.astype(int)
    x = copy.deepcopy(heatmap)
    r_temp = img[:,:,0]*(x*(-1))
    img[:,:,0] = img[:,:,0] +r_temp +(0*x)
    g_temp = img[:,:,1]*(x*(-1))
    img[:,:,1] = img[:,:,1] +g_temp +(0*x)
    b_temp = img[:,:,2]*(x*(-1))
    img[:,:,2] = img[:,:,2] +b_temp +(0*x)
    '''
    '''
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    print("check the name")
    print(name)
    
    cv2.imwrite('./test1/'+str(name)+'.jpg', img)
    #plt.imshow(img)
    #break
    #########################################################################for label
   '''

'\nheatmap = np.uint8(255 * heatmap)\nheatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)\nimg = heatmap * 0.4 + img\nname = path[0].split(\'/\')[-1].split(\'.\')[-2]\nprint("check the name")\nprint(name)\n\ncv2.imwrite(\'./test1/\'+str(name)+\'.jpg\', img)\n#plt.imshow(img)\n#break\n#########################################################################for label\n   '

for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    img1 = data['img'].to(DEVICE)
    print("check the image")
    print(img1.shape)
    path = data['path']

    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    print(img1.shape)
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    print("CHECK THE PATH")
    print(path[0])   
    resnet = ResNet()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    print("check the image sizes")
    print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred, acts = resnet(img1)
    max_index = pred.argmax(dim=1) 
    print("check the max index")
    print(max_index)
    pred[:,(max_index.item())].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    
    print(np.unique(heatmap))
    
    print(label.item())
    img = io.imread(path[0])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    p = np.percentile(heatmap, 70)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    cv2.imwrite('./originallabel_camfiles/'+str(name)+'_'+str(max_index.item()+1)+'.jpg', superimposed_img)
    

#plt.imshow(data["img"])
for data in testloader:
    #net1 = net1.to(DEVICE)
    #net2 = net2.to(DEVICE)
    img1 = data['img'].to(DEVICE)
    print("check the image")
    print(img1.shape)
    path = data['path']

    #img1 = img1.float()
    img1 = img1.view(-1,img1.shape[-3],img1.shape[-2],img1.shape[-1])
    print(img1.shape)
    img1 = img1.float()
    #print(img1.shape)
    label = data['label'].to(DEVICE)
    mislabel = data['mislabel'].to(DEVICE)
   
    resnet = ResNet()
    resnet = resnet.to(DEVICE)
    resnet.eval()
    print("check the image sizes")
    print(img1.shape)
    img1 = Variable(img1.cuda(), requires_grad=True)
    pred, acts = resnet(img1)
    max_index = pred.argmax(dim=1) 
    print("check the max index")
    print(max_index)
    ##############################################################for actual label
    pred[:,(label.item())].backward()
    gradients = resnet.get_gradient()
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activations = resnet.get_activations(img1).detach()
    for i in range(activations.shape[1]):
        activations[:, i, :, :] *= pooled_gradients[i]
    heatmap = F.relu(torch.sum(activations, dim=1)).squeeze()
    heatmap = np.maximum(heatmap.cpu(), 0)
    heatmap /= torch.max(heatmap)
    #plt.matshow(heatmap.squeeze())
    heatmap = heatmap.numpy()
    #print(np.unique(heatmap))
    p = np.percentile(heatmap, 97)
    heatmap[heatmap>=p]=1
    heatmap[heatmap<p]=0
    print(np.unique(heatmap))
    print("CHECK THE PATH")
    print(path[0])
    print(label.item())
    img = io.imread(path[0])
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img
    name = path[0].split('/')[-1].split('.')[-2]
    print("check the name")
    print(name)
    
    cv2.imwrite('./originallabel_camfiles/'+str(name)+'_'+str(label.item())+'.jpg', superimposed_img)
    #########################################################################for label
   

In [ ]:
print(list_image)

In [297]:
with open('./actualimagefile.txt', 'w') as filehandle:
    for i in range(len(list_image)):
        filehandle.write('%s|%s\n' % (list_image[i][0],list_label[i]))